In [99]:
import sys
import pandas as pd
from  random import randint
sys.path.append("..")
from next_word_prediction import GPT2
from gpt_2.src.interactive_conditional_samples import interact_model
from sklearn.metrics import accuracy_score
from IPython.display import clear_output
clear_output(wait=True)

In [100]:
gpt2 = GPT2()

In [101]:
data_base = pd.read_csv('../swiss-test/data/dataset_base.csv')

data_no_location = pd.read_csv('../swiss-test/data/dataset_no_location.csv')

In [102]:
after_sentences = [ ' All this happens in the ', ' It was great time in the ', ' Hope to see again this ', ' This place is located in the ']

In [103]:
X_base = data_base['text']
X_base = list(map(lambda text: text + after_sentences[randint(0, len(after_sentences) - 1)], X_base))
y_base = data_base['loc'].astype(str)

X_no = data_no_location['text']
X_no = list(map(lambda text: text + after_sentences[randint(0, len(after_sentences) - 1)], X_no))
y_no = data_no_location['loc'].astype(str)

In [104]:
def get_random_sentences(sentences, n=3):
    result = []
    for _ in range(n):
        result.append(sentences[randint(0, len(sentences) - 1)])
    return result

def predict(y_true, y_pred):
    result = []
    for y_t, y_p in list(zip(y_true, y_pred)):
        if y_t in y_p:
            result.append(y_t)
        else:
            result.append(y_p[0])
    return result

In [105]:
def generate_sentences(sentences):
    generated = []
    for _ in range(len(sentences)):
        generated.append(interact_model(raw_text=sentences))
    return generated

def predict_next_words(text, n=10):
    return gpt2.predict_next(text, n)

def predict_words(sentences, n=10):
    return list(map(lambda text: gpt2.predict_next(text, n), sentences))

In [106]:
base_random = get_random_sentences(list(zip(X_base, y_base)))
no_random = get_random_sentences(list(zip(X_no, y_no)))

In [107]:
for (X_i_base, y_i_base) in base_random:
    predicted_words = predict_next_words(X_i_base)
    print('Original text: {}'.format(X_i_base))
    print('True location: {}'.format(y_i_base))
    print('Predicted locations: {}'.format(predicted_words))
    print('Prediction success: {}'.format(y_i_base in predicted_words))
    print('=' * 40)

Original text: Never was their a picture as enchanting as the ever-evolving sea. All this happens in the 
True location: sea
Predicted locations: ['same', 'midst', 'course', 'middle', 'face', 'form', 'ocean', 'name', 'wake', 'world']
Prediction success: False
Original text: The buildings galloped up the clouds and they had entire floors dedicated to play. There were slides to go down several floors at a time, they had elevators too but fun was thought so important that those were added too.  Hope to see again this 
True location: modern building
Predicted locations: ['year', 'summer', 'weekend', 'week', 'time', 'fall', 'afternoon', 'evening', 'season', 'morning']
Prediction success: False
Original text: The city was a vast, intricate, labyrinth of noisy, streets and alleys. You could hear the incessant honking of the vehicles even before the sun rose. Impatient businessmen who had to get to the office, mothers who had to leave their children at their schools, facing the puzzling challe

In [108]:
for (X_i_no, y_i_no) in no_random:
    predicted_words = predict_next_words(X_i_no)
    print('Original text: {}'.format(X_i_no))
    print('True location: {}'.format(y_i_no))
    print('Predicted locations: {}'.format(predicted_words))
    print('Prediction success: {}'.format(y_i_no in predicted_words))
    print('=' * 40)

Original text: Upon this primrose sand, the hue as gentle on the eye as a vintage photograph, there is a steady warmth from the grains. Already the stars glow as if they have kept a pocket of the daytime to shine all through the night sky. Sometimes I think the earth and the moon choose to give of their borrowed warmth and light until the return of the sun, the brilliance forever promised at dawn.  This place is located in the 
True location: beach
Predicted locations: ['middle', 'heart', 'center', 'footh', 'midst', 'northern', 'southern', 'mountains', 'beautiful', 'shadow']
Prediction success: False
Original text: Her eyes began to water as a wind whipped the hair across her face. When she looked down at the dizzying drop, nothing met her gaze but flat clouds, hiding the treasure of their landscape below. All this happens in the 
True location: mountain
Predicted locations: ['same', 'blink', 'middle', 'night', 'dark', 'midst', 'distance', 'moment', 'background', 'world']
Prediction su

In [109]:
score = accuracy_score(y_true=y_base, y_pred=predict(y_base, predict_words(X_base)))
accuracy = '{}%'.format(int(100. * score))

print('Accuracy for sentences with location: {}'.format(accuracy))

score = accuracy_score(y_true=y_no, y_pred=predict(y_no, predict_words(X_no)))
accuracy = '{}%'.format(int(100. * score))

print('Accuracy for sentences without location: {}'.format(accuracy))

Accuracy for sentences with location: 28%
Accuracy for sentences without location: 11%


In [110]:
predicted_texts = []

for (X_i_base, _) in base_random:
    predicted_text = interact_model(raw_text=X_i_base)
    predicted_texts.append(predicted_text)
    clear_output(wait=True)

for i in range(len(base_random)):
    X_i_base, y_i_base = base_random[i]
    predicted_text = predicted_texts[i]
    print('Original text: {}'.format(X_i_base))
    print('True location: {}'.format(y_i_base))
    print('Full predicted text: \n{} {}'.format(X_i_base, predicted_text))
    print('=' * 40)

Original text: Never was their a picture as enchanting as the ever-evolving sea. All this happens in the 
True location: sea
Full predicted text: 
Never was their a picture as enchanting as the ever-evolving sea. All this happens in the  vernacular. I suspect everything that means or means is already being written in that language, but if we will plead against the world's antagonist, then our world must have its day. Don't forget to side with humanity, and even give people to your cause. Ask for the justice that backs you on what is reckoned already fifteen billion years, and it will come. If we don't do this, nationalism isn't going to be for our harms. Listening to Occupy California, I could hear Twin Peaks and critics of Israel talking in tongues.

NEAR THESE STRIVE OR NOT:

As you can see from Seattleites, people are also saying I am arrogant, unrestrained, unqualified for world leadership. I am no Army officer, I passed 119 attempts, I wear an expensive baseball cap and star embro

In [111]:
predicted_texts = []

for (X_i_no, _) in base_random:
    predicted_text = interact_model(raw_text=X_i_no)
    predicted_texts.append(predicted_text)
    clear_output(wait=True)

for i in range(len(no_random)):
    X_i_no, y_i_no = no_random[i]
    predicted_text = predicted_texts[i]
    print('Original text: {}'.format(X_i_no))
    print('True location: {}'.format(y_i_no))
    print('Full predicted text: \n{} {}'.format(X_i_no, predicted_text))
    print('=' * 40)

Original text: Upon this primrose sand, the hue as gentle on the eye as a vintage photograph, there is a steady warmth from the grains. Already the stars glow as if they have kept a pocket of the daytime to shine all through the night sky. Sometimes I think the earth and the moon choose to give of their borrowed warmth and light until the return of the sun, the brilliance forever promised at dawn.  This place is located in the 
True location: beach
Full predicted text: 
Upon this primrose sand, the hue as gentle on the eye as a vintage photograph, there is a steady warmth from the grains. Already the stars glow as if they have kept a pocket of the daytime to shine all through the night sky. Sometimes I think the earth and the moon choose to give of their borrowed warmth and light until the return of the sun, the brilliance forever promised at dawn.  This place is located in the  vernal sky, which is spreading towards us as flickering orange stars moving slowly in our path.
Every seed o